In [10]:
import json
from OTXv2 import OTXv2, IndicatorTypes
import requests

# Function to load API key from a config file
def load_api_key(config_path):
    with open(config_path, "r") as file:
        config = json.load(file)
        return config.get("API_KEY")

# API Configuration
CONFIG_FILE = "config.json"  # Path to your JSON file with API key
API_KEY = load_api_key(CONFIG_FILE)
OTX_SERVER = 'https://otx.alienvault.com/'
otx = OTXv2(API_KEY, server=OTX_SERVER)

# Fetch Pulse Details (670f94e03014212e19fa5a77)
pulse_id = "67429f73a3f45fa88890276d"
pulse_details = otx.get_pulse_details(pulse_id)

# Extract the indicators from the pulse details
indicators = pulse_details.get("indicators", [])
print(f"Total indicators found in pulse: {len(indicators)}")

Total indicators found in pulse: 14883


In [ ]:
import csv

# Initialize a list to store indicator details
fetched_data = []

# Counter for limiting to 100 indicators
counter = 0

# Loop through the indicators
for indicator_data in indicators:
    if counter >= 100:
        break  # Stop after fetching 100 indicators

    indicator_value = indicator_data.get("indicator")
    indicator_type = indicator_data.get("type")
    
    if indicator_value and indicator_type:
        print(f"Fetching details for indicator: {indicator_value} (Type: {indicator_type})")
        
        # Convert the indicator type to the correct OTX IndicatorTypes
        if indicator_type.lower() == "ipv4":
            indicator_type_otx = IndicatorTypes.IPv4
        elif indicator_type.lower() == "domain":
            indicator_type_otx = IndicatorTypes.DOMAIN
        elif indicator_type.lower() == "url":
            indicator_type_otx = IndicatorTypes.URL
        elif indicator_type.lower() == "file_hash_md5":
            indicator_type_otx = IndicatorTypes.FILE_HASH_MD5
        else:
            print(f"Unsupported indicator type: {indicator_type}")
            continue
        
        # Fetch full details of the indicator
        indicator_details = otx.get_indicator_details_full(indicator_type_otx, indicator_value)
        
        # Append the details to the list
        fetched_data.append({
            "indicator": indicator_value,
            "type": indicator_type,
            "details": indicator_details  # This could be serialized data; modify based on actual content
        })
        
        counter += 1

# Write the data to a CSV file
csv_file = "indicators_details.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["indicator", "type", "details"])
    writer.writeheader()
    writer.writerows(fetched_data)

print(f"Saved the first 100 indicator details to {csv_file}")


In [ ]:
# extract a given indicator's analysis overview
import requests

def get_facts(type, ip):
    response = requests.get(f'https://otx.alienvault.com/otxapi/indicators/ip/analysis/{ip}')
    print(response.json()['facts'])

get_facts('8.8.8.8')

{'verdict': 'Whitelisted', 'reverse_dns': 'dns.google', 'otx_telemetry_7_days': True, 'otx_telemetry_30_days': True, 'otx_telemetry_all': True, 'unique_cves_7_days': ['2015-5119', '2015-7547'], 'unique_cves_30_days': ['2017-0144'], 'unique_cves_all': [], 'indicator_popularity': 0, 'ssl_certificates': [{'port': 443, 'subject': 'CN=dns.google', 'issuer': 'C=US, O=Google Trust Services, CN=WR2', 'ja3': '771,49199,65281', 'fingerprint': '72:b5:bf:c1:6e:12:79:2c:b9:db:de:e1:17:11:3d:05:08:1d:48:f5:a1:ea:af:d1:90:20:d5:c8:fc:c9:e4:6b'}, {'port': 443, 'subject': 'CN=dns.google', 'issuer': 'C=US, O=Google Trust Services, CN=WR2', 'ja3': '771,49199,65281', 'fingerprint': '72:b5:bf:c1:6e:12:79:2c:b9:db:de:e1:17:11:3d:05:08:1d:48:f5:a1:ea:af:d1:90:20:d5:c8:fc:c9:e4:6b'}], 'number_of_domains_resolving_7_days': 56, 'number_of_domains_resolving_30_days': 295, 'number_of_domains_resolving_all': 500, 'unique_tlds_from_domains_resolving': 98, 'number_of_dynamic_dns_in_pdns': 1, 'url_indicators_from_the

In [16]:
indicators_list = indicators

In [14]:
indicators

[{'id': 3888502107,
  'indicator': '2c0b9017cf55630f1095ff42d9717732',
  'type': 'FileHash-MD5',
  'created': '2024-11-24T03:37:23',
  'content': '',
  'title': '',
  'description': '',
  'expiration': None,
  'is_active': 1},
 {'id': 3888502108,
  'indicator': '4e09ef9806fb9af448a5efcd60395815',
  'type': 'FileHash-MD5',
  'created': '2024-11-24T03:37:23',
  'content': '',
  'title': '',
  'description': '',
  'expiration': None,
  'is_active': 1},
 {'id': 3888502109,
  'indicator': '50e5927056538d5087816be6852397f6',
  'type': 'FileHash-MD5',
  'created': '2024-11-24T03:37:23',
  'content': '',
  'title': '',
  'description': '',
  'expiration': None,
  'is_active': 1},
 {'id': 3888502110,
  'indicator': '6e4ae1209a2afe123636f6074c19745d',
  'type': 'FileHash-MD5',
  'created': '2024-11-24T03:37:23',
  'content': '',
  'title': '',
  'description': '',
  'expiration': None,
  'is_active': 1},
 {'id': 3914811965,
  'indicator': '952bb88b4ce8c402b25d219ea27ad9e4',
  'type': 'FileHash-M

In [ ]:
#working
import requests
import json

# Function to fetch facts for an indicator and construct the appropriate URL
def get_facts(indicator_value, indicator_type):
    # Base URL for OTX indicators with the new format
    base_url = "https://otx.alienvault.com/otxapi/indicators"

    # Map indicator types to their URL path, handle "file" separately
    type_to_url_path = {
        "url": "url",
        "domain": "domain",
        "file": "file",  # Always use "file" for file-related types
        "ipv4": "ip",    # For IPv4, use "ip"
        "hostname": "hostname",
    }

    # Get the specific path for the given type, default to "file" if it's a file-related indicator
    if indicator_type.lower() in ["file_hash_md5", "file_hash_sha1", "file_hash_sha256", "file_hash_imphash", "file_hash_pehash"]:
        url_path = "file"
    else:
        url_path = type_to_url_path.get(indicator_type.lower())

    if not url_path:
        print(f"Unsupported type: {indicator_type}")
        return {}

    indicator_url = f"{base_url}/{url_path}/analysis/{indicator_value}"
    print(f"Fetching from URL: {indicator_url}")

    # Perform the request
    response = requests.get(indicator_url)
    if response.status_code == 200:
        try:
            # Try to parse JSON response
            return response.json()
        except ValueError as e:
            print(f"Error decoding JSON for {indicator_value} (Type: {indicator_type}): {e}")
            return None
    else:
        print(f"Failed to fetch details for {indicator_value} (Type: {indicator_type}): {response.status_code}")
        return None

# Sample list of indicators
indicators = [
    {"indicator": "http://164.90.244.118/", "type": "url"},
    {"indicator": "strikebripm.sbs", "type": "domain"},
    {"indicator": "ecae71c0b822df22ab7be6cbc659bc24", "type": "file_hash_md5"},
    {"indicator": "8.8.8.8", "type": "ipv4"}
]

# Initialize a list to store fetched indicator details
fetched_data = []

# Process the first 100 indicators
counter = 0
for indicator_data in indicators:
    if counter >= 100:  # Limit to the first 100 indicators
        break

    indicator_value = indicator_data.get("indicator")
    indicator_type = indicator_data.get("type")

    if indicator_value and indicator_type:
        print(f"Fetching details for indicator: {indicator_value} (Type: {indicator_type})")
        
        # Fetch full details of the indicator
        indicator_details = get_facts(indicator_value, indicator_type)
        
        if indicator_details:
            # Store the indicator, type, and full details
            fetched_data.append({
                "indicator": indicator_value,
                "type": indicator_type,
                "details": indicator_details
            })
        counter += 1

# Write the fetched data to a JSON file
json_file = "fetched_indicators_details.json"
with open(json_file, mode="w", encoding="utf-8") as file:
    json.dump(fetched_data, file, indent=4)

print(f"Saved the first 100 indicator details to {json_file}")


Fetching details for indicator: http://164.90.244.118/ (Type: url)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/url/analysis/http://164.90.244.118/
Fetching details for indicator: strikebripm.sbs (Type: domain)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/domain/analysis/strikebripm.sbs
Fetching details for indicator: ecae71c0b822df22ab7be6cbc659bc24 (Type: file_hash_md5)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/file/analysis/ecae71c0b822df22ab7be6cbc659bc24
Fetching details for indicator: 8.8.8.8 (Type: ipv4)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/ip/analysis/8.8.8.8
Saved the first 100 indicator details to fetched_indicators_details.json


In [ ]:
# partially working
import requests
import json

# Function to fetch facts for an indicator and construct the appropriate URL
def get_facts(indicator_value, indicator_type):
    # Base URL for OTX indicators with the new format
    base_url = "https://otx.alienvault.com/otxapi/indicators"

    # Map indicator types to their URL path
    type_to_url_path = {
        "url": "url",
        "domain": "domain",
        "file": "file",
        "ipv4": "ip",
        "hostname": "hostname",
    }

    # Get the specific path for the given type
    url_path = type_to_url_path.get(indicator_type.lower())
    if not url_path:
        print(f"Unsupported type: {indicator_type}")
        return {}
    
    indicator_url = f"{base_url}/{indicator_type}/analysis/{indicator_value}"
    print(f"Fetching from URL: {indicator_url}")

    # Perform the request
    response = requests.get(indicator_url)
    if response.status_code == 200:
        try:
            # Try to parse JSON response
            return response.json()
        except ValueError as e:
            print(f"Error decoding JSON for {indicator_value} (Type: {indicator_type}): {e}")
            return None
    else:
        print(f"Failed to fetch details for {indicator_value} (Type: {indicator_type}): {response.status_code}")
        return None

# Sample list of indicators
indicators = [
    {"indicator": "http://164.90.244.118/", "type": "url"},
    {"indicator": "strikebripm.sbs", "type": "domain"},
    {"indicator": "ecae71c0b822df22ab7be6cbc659bc24", "type": "file_hash_md5"},
    {"indicator": "8.8.8.8", "type": "ipv4"}
]

# Initialize a list to store fetched indicator details
fetched_data = []

# Process the first 100 indicators
counter = 0
for indicator_data in indicators:
    if counter >= 100:  # Limit to the first 100 indicators
        break

    indicator_value = indicator_data.get("indicator")
    indicator_type = indicator_data.get("type")

    if indicator_value and indicator_type:
        print(f"Fetching details for indicator: {indicator_value} (Type: {indicator_type})")
        
        # Fetch full details of the indicator
        indicator_details = get_facts(indicator_value, indicator_type)
        
        if indicator_details:
            # Store the indicator, type, and full details
            fetched_data.append({
                "indicator": indicator_value,
                "type": indicator_type,
                "details": indicator_details
            })
        counter += 1

# Write the fetched data to a JSON file
json_file = "fetched_indicators_details.json"
with open(json_file, mode="w", encoding="utf-8") as file:
    json.dump(fetched_data, file, indent=4)

print(f"Saved the first 100 indicator details to {json_file}")


Fetching details for indicator: http://164.90.244.118/ (Type: url)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/url/analysis/http://164.90.244.118/
Fetching details for indicator: strikebripm.sbs (Type: domain)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/domain/analysis/strikebripm.sbs
Fetching details for indicator: ecae71c0b822df22ab7be6cbc659bc24 (Type: file_hash_md5)
Unsupported type: file_hash_md5
Fetching details for indicator: 8.8.8.8 (Type: ipv4)
Fetching from URL: https://otx.alienvault.com/otxapi/indicators/ipv4/analysis/8.8.8.8
Failed to fetch details for 8.8.8.8 (Type: ipv4): 404
Saved the first 100 indicator details to fetched_indicators_details.json


In [ ]:
# Check each indicator and fetch its details
for indicator_data in indicators:
    indicator_value = indicator_data.get("indicator")
    indicator_type = indicator_data.get("type")
    
    if indicator_value and indicator_type:
        print(f"Fetching details for indicator: {indicator_value} (Type: {indicator_type})")
        
        # Convert the indicator type to the correct OTX IndicatorTypes (Handle different indicator types)
        if indicator_type.lower() == "ipv4":
            indicator_type_otx = IndicatorTypes.IPv4
        elif indicator_type.lower() == "domain":
            indicator_type_otx = IndicatorTypes.DOMAIN
        elif indicator_type.lower() == "url":
            indicator_type_otx = IndicatorTypes.URL
        elif indicator_type.lower() == "file_hash_md5":
            indicator_type_otx = IndicatorTypes.FILE_HASH_MD5
        else:
            print(f"Unsupported indicator type: {indicator_type}")
            continue
        
        # Fetch full details of the indicator
        indicator_details = otx.get_indicator_details_full(indicator_type_otx, indicator_value)